In [ ]:
import os
import cv2
import re
import numpy as np
from PIL import Image
import pytesseract
import logging
import paramiko
import time
from scp import SCPClient

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Set path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to preprocess the image
def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        resized_image = cv2.resize(thresh_image, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
        denoised_image = cv2.fastNlMeansDenoising(resized_image, None, 30, 7, 21)
        preprocessed_image_path = "preprocessed_image.png"
        cv2.imwrite(preprocessed_image_path, denoised_image)
        return preprocessed_image_path
    except Exception as e:
        logger.error(f"Error preprocessing image {image_path}: {e}")
        return None

# Function to extract passwords from an image
def extract_password_from_image(image_path, keywords):
    try:
        preprocessed_image_path = preprocess_image(image_path)
        if not preprocessed_image_path:
            return None
        
        image = Image.open(preprocessed_image_path)
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(image, config=custom_config)
        text = text.replace('0', 'O')
        pattern = re.compile(r'\b[A-Z0-9]{10}\b')
        lines = text.split('\n')
        for line in lines:
            for keyword in keywords:
                if keyword.lower() in line.lower():
                    match = pattern.search(line)
                    if match:
                        password = match.group(0)
                        return password
        return None
    except FileNotFoundError:
        logger.error(f"File not found: {image_path}")
    except IOError:
        logger.error(f"IO error with file: {image_path}")
    except Exception as e:
        logger.error(f"Error processing file {image_path}: {e}")
    return None

# Function to process images and extract passwords
def process_images(image_paths, keywords):
    passwords = []
    for path in image_paths:
        if os.path.exists(path):
            password = extract_password_from_image(path, keywords)
            if password:
                passwords.append(password)
        else:
            logger.error(f"Image path does not exist: {path}")
    return passwords

# List of image paths and keywords
image_paths = [
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.59.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.58.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.57.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.56.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.55 (1).jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.55.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.54 (1).jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.54.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.05 (1).jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.05.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.04 (1).jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.04.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.03 (1).jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.03.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.02 (1).jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.02.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.01.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.00 (1).jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.43.00.jpeg",
r"C:\Users\Administrator\Downloads\WhatsApp Image 2024-06-14 at 14.42.59 (1).jpeg"
]

keywords = ["Password"]
passwords = process_images(image_paths, keywords)

# Print extracted passwords
if passwords:
    logger.info("Extracted passwords:")
    for password in passwords:
        print(password)
else:
    logger.info("No passwords extracted from any images.")


In [ ]:
print(password)

In [ ]:
import paramiko
import time
from scp import SCPClient
import re

# Fungsi untuk membuat koneksi SSH
def create_ssh_client(host, port, username, password):
    try:
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(host, port, username, password)
        print(f"SSH connection established successfully with password: {password}")
        return client
    except Exception as e:
        print(f"Error establishing SSH connection with password {password}: {e}")
        return None

# Fungsi untuk mengupload file menggunakan SCP
def upload_file(ssh_client, local_file, remote_path):
    try:
        with SCPClient(ssh_client.get_transport()) as scp:
            scp.put(local_file, remote_path)
        print("File uploaded successfully.")
    except Exception as e:
        print(f"Error uploading file: {e}")

# Fungsi untuk mengeksekusi perintah di MikroTik
def execute_command(ssh_client, command):
    try:
        stdin, stdout, stderr = ssh_client.exec_command(command)
        output = stdout.read().decode()
        print(f"Command executed successfully: {command}")
        return output
    except Exception as e:
        print(f"Error executing command '{command}': {e}")
        return None

# Fungsi untuk mengekstrak nilai serial number dari teks
def extract_serial_number(resource_output):
    match = re.search(r"serial number\s*=\s*(\w+)", resource_output, re.IGNORECASE)
    if match:
        return match.group(1)
    return None

# Detail koneksi
host = "192.168.88.1"  # Ganti dengan IP MikroTik Anda
port = 22
username = "admin"  # Ganti dengan username MikroTik Anda
local_file = "D:\\temp\\customblock-ax2.npk"  # Ganti dengan path file lokal Anda
remote_path = "/customblock-ax2.npk"  # Ganti dengan path remote di MikroTik

# Mencoba koneksi SSH dengan setiap password
ssh_client = None
successful_password = None
for password in passwords:
    ssh_client = create_ssh_client(host, port, username, password)
    if ssh_client:
        successful_password = password
        break

if ssh_client:
    upload_file(ssh_client, local_file, remote_path)

    # Melakukan reboot perangkat
    execute_command(ssh_client, "/system reboot")

    # Menutup koneksi SSH
    ssh_client.close()
    print("SSH connection closed.")

    # Menunggu perangkat reboot (ini hanya estimasi, sesuaikan dengan waktu reboot perangkat Anda)
    time.sleep(60)

    # Mencoba koneksi SSH kembali dengan password yang berhasil dari login pertama
    if successful_password:
        ssh_client = create_ssh_client(host, port, username, successful_password)
        if ssh_client:
            # Mendapatkan nilai dari perintah /system resource print
            resource_output = execute_command(ssh_client, "/system/resource export")
            serial_number = extract_serial_number(resource_output)
            
            if serial_number:
                # Menyimpan nilai serial number ke file teks
                with open("mikrotik_info.txt", "a") as file:
                    file.write(f"\n{serial_number}")
                
                print("Serial number retrieved and saved successfully.")
            
            # Menutup koneksi SSH
            execute_command(ssh_client, "/ip arp remove [find]")
            ssh_client.close()
            print("SSH connection closed.")
        else:
            print("Failed to re-establish SSH connection after reboot.")
else:
    print("Failed to establish initial SSH connection. Exiting.")